In [18]:
import re
import pandas as pd
import joblib
df=pd.read_csv("../Data/f1_dataset.csv")
df.head()
df.info()
df.isna().sum()
df['winner_name'].value_counts()

def clean_text(text):
    text=text.lower()
    text=re.sub(r"\s+"," ",text)
    text=re.sub(r"\[[0-9]+\]","",text)
    return text.strip()

df["clean_text"]=df["full_text"].apply(clean_text)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   race_id      81 non-null     int64 
 1   race_name    81 non-null     object
 2   race_date    81 non-null     object
 3   race_year    81 non-null     int64 
 4   winner_name  81 non-null     object
 5   winner_team  79 non-null     object
 6   full_text    81 non-null     object
dtypes: int64(2), object(5)
memory usage: 4.6+ KB


In [19]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["label"]=le.fit_transform(df["winner_name"])
joblib.dump(le, "models/label_encoder.pkl")
print(list(le.classes_))

['Charles Leclerc', 'George Russell', 'Lando Norris', 'Lewis Hamilton', 'Max Verstappen', 'Oscar Piastri', 'Pierre Gasly', 'Sergio Pérez', 'Valtteri Bottas']


In [20]:
print(df["winner_name"].value_counts())

winner_name
Lewis Hamilton     28
Max Verstappen     27
Lando Norris        9
Oscar Piastri       5
Sergio Pérez        5
Charles Leclerc     3
George Russell      2
Pierre Gasly        1
Valtteri Bottas     1
Name: count, dtype: int64


In [21]:
counts = df['label'].value_counts()
rare = counts[counts == 1].index
rare


Index([6, 8], dtype='int64', name='label')

In [22]:
for r in rare:
    df = pd.concat([df, df[df['label'] == r]])


In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df["clean_text"],
    df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]     
)


In [24]:
import os 
os.makedirs("Data/Processed",exist_ok=True)
train_df=pd.DataFrame({"text": X_train, "label": y_train})
text_df=pd.DataFrame({"text": X_test, "label": y_test})

train_df.to_csv("Data/Processed/train.csv",index=False)
text_df.to_csv("Data/Processed/test.csv",index=False)   
print("Preprocessed data saved to Data/Processed/train.csv and Data/Processed/test.csv")

Preprocessed data saved to Data/Processed/train.csv and Data/Processed/test.csv
